
Author: Gickel OKABI

Python developer



.

customer's request : In this project you will extract a quantity of 1081 recipes from https://www.myplate.gov/myplate-kitchen/recipes. The recipes will be formatted into a single JSON or csv file containing a single recipe with fields: TItle, Image, Ingredients, Directions, Notes, and Source. The "Source" value will be "USDA" and the "Image" value will be a URL to the image for the recipe.

In [1]:
# Import necessary libraries

import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

 .

The first function allows us to collect the urls of the pages we want


In [2]:
def page_link(first_page,last_page):
    urls = []
    for i in range(first_page,1+last_page):
        url = f"https://www.myplate.gov/myplate-kitchen/recipes?sort_bef_combine=title_ASC&page={i}"
        urls.append(url)
    return urls

.

The second function takes the urls of the pages as parameters and returns the urls of each element


In [3]:
def get_all_link(pages_links):

    all_recipe = []
    
    #loop through all pages
    for page_link in pages_links:
        page_contenu = requests.get(page_link)
        soup = BeautifulSoup(page_contenu.content, 'html.parser')
        recipes = soup.find_all("div", class_="views-row") 
        for recipes_case in recipes:
            recipe_link = recipes_case.find("div", class_="mp-recipe-teaser__title").find('a')['href']
            recipe_link = "".join(["https://www.myplate.gov",recipe_link ])
            all_recipe.append(recipe_link)
            
    return all_recipe        

.

The third function takes the element's link as a parameter to extract all the characteristics requested by the client


In [4]:
def data_collecte(links):
    titles = []
    ingredients= []
    directions = []
    notes = []
    sources = []
    images  = []
    
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    }
    
    for link in links:
        
        response = requests.get(link,headers=headers)


        soup = BeautifulSoup(response.content, 'html.parser')


        # extract title
        try:
            title = soup.find('span', class_=["field field--name-title field--type-string field--label-hidden"]).text
        except :
            title = ""


        # extract title
        try:
            ingred_ul = soup.find('ul', class_=["field__items ingredients yyyyy"])
            ingredient = [re.sub(r"\s+"," ",ingredien.text.strip()) for ingredien in ingred_ul.find_all('li', class_="field__item")]
        except:
            ingredient = ""

        # extract directions
        try:
            direction = [direction.text.strip() for direction in soup.select('div.clearfix.text-formatted.field.field--name-field-instructions.field--type-text-long.field--label-above div.field__item ol li')]
        except:
            direction = ""


        # extract note
        try:
            note = soup.find('div', class_= ['clearfix text-formatted field field--name-field-notes field--type-text-long field--label-above']).find('div',class_='field__item').text.strip()
        except:
            note =""
        # extract source
        source = "USBA"

        # extract image url
        try:
            image = soup.find('div', class_ = ["field field--name-field-recipe-image field--type-image field--label-visually_hidden"]).find('img')['src']
        except:
            image=""
        
        # Save items to list
        titles.append(title)
        images.append(image)
        ingredients.append(ingredient)
        directions.append(direction)
        notes.append(note)
        sources.append(source)
        
    #creation of the data frame    
    dataframe = pd.DataFrame({
                'title' : titles,
                'ingredients':ingredients,
                'directions' : directions,
                'notes' :notes,
                'sources' : sources,
                'image' : images  
            })        
        
    return dataframe   

.

function "data_collecte" takes as parameter the list of element links returned by function "get_all_link" which itself takes as parameter the list of page links returned by function "page_link"


In [5]:
dataframe = data_collecte(get_all_link(page_link(1,54)))

.

Display the first 10 rows of all collected data


In [6]:
dataframe.head(10)

,title,ingredients,directions,notes,sources,image
0,Apple Cranberry Salad Toss,"[1 head of lettuce (about 10 cups), 2 apples (...","[Wash hands with soap and water., Toss lettuce...",Learn more about:\nLettuce\nApples\nCranberrie...,USBA,https://myplate-prod.azureedge.us/sites/defaul...
1,Apple Crisp,"[4 apples (medium), 1/4 cup oatmeal (quick coo...","[Preheat the oven to 350 °F., Grease the botto...","For different tastes, try other fruits like ch...",USBA,https://myplate-prod.azureedge.us/sites/defaul...
2,Apple Oatmeal Muffins,"[1/2 cup milk, non-fat, 1/3 cup applesauce, 1/...","[Wash hands with soap and water., Preheat oven...","Optional toppings: raisins, walnuts.\nLearn mo...",USBA,https://myplate-prod.azureedge.us/sites/defaul...
3,Apple Pistachio Crisp,"[3 apples (such as Honeycrisp, cored and cut i...",[Place rack in center of oven and preheat to 3...,Serving Suggestion: Serve with an 8 oz. glass ...,USBA,https://myplate-prod.azureedge.us/sites/defaul...
4,Apple Salad,"[2 cups apple (diced), 1 cup celery (diced), 1...",[Mix orange juice with salad dressing or mayon...,Learn more about:\nApples\nCelery,USBA,https://myplate-prod.azureedge.us/sites/defaul...
5,Apple Salad with Sweet & Tangy Yogurt Dressing,"[1 cup diced apple, 1 teaspoon lemon juice, 1/...","[Wash hands with soap and water., Wash apples,...",Nutrition analysis was done using 1/2 cup low-...,USBA,https://myplate-prod.azureedge.us/sites/defaul...
6,Apple Sandwiches,"[1 medium apple, 2 tablespoons peanut butter, ...","[Wash hands with soap and water., Cut apple in...","Sprinkle the peanut butter with grated carrot,...",USBA,https://myplate-prod.azureedge.us/sites/defaul...
7,Apple Slice Pancakes,"[1 apple (Granny Smith), 1 1/4 cups pancake mi...",[Lightly coat a griddle or skillet with cookin...,"To test the griddle to see if it is hot, sprin...",USBA,https://myplate-prod.azureedge.us/sites/defaul...
8,Apple Tuna Sandwiches,"[1 can tuna, packed in water (6.5 ounces, drai...","[Wash hands with soap and water., Wash and pee...",Learn more about:\nLettuce\nApples,USBA,https://myplate-prod.azureedge.us/sites/defaul...
9,Apple Wedges with Pumpkin Almond Butter,[],"[Mix together dip ingredients., Serve with app...",Learn more about:\nApples,USBA,https://myplate-prod.azureedge.us/sites/defaul...


.

In [ ]:
# Exporter les données au format CSV
database.to_csv("donnees.csv", index=False)

.

Thanks for looking through the notebook.

If you have a data collection project, do not hesitate to entrust it to me. I will provide you with both the code and the collected dataset.

Gickel OKABI